# Exploring ipyvolume

[ipyvolume](https://ipyvolume.readthedocs.io/en/latest/index.html) may help explore differences in tile image pixel values.

**Note:**  I'm using macOS 11.2.3 with Anaconda python 3.9.2 and am finding that ipyvolume 0.5.2 can display quiver plots in `jupyter notebook`, but not in `jupyter lab`.


In [1]:
import ipyvolume
ipyvolume.__version__

'0.5.2'

In [2]:
from pathlib import Path

import numpy as np
from skimage import color
from skimage.util import img_as_ubyte
import ipyvolume as ipv

from band_finder.image_db import ImageDB
from band_finder.image_cache import ImageCache
from band_finder.bayer_to_rgb import bayer_to_rgb
from band_finder.tile_image_grid import TileImageGrid, Edge

In [3]:
examples_dir = Path("../examples").resolve()

db_path = list(examples_dir.glob("*.db"))[0]
assert db_path.exists()

image_db = ImageDB(db_path)

cache_dir = examples_dir / "image_cache"
assert cache_dir.exists()

image_cache = ImageCache(image_db, cache_dir)


def get_rgb_image(image_id):
    image = image_cache.get_image(image_id)
    # If it is a bayer image, convert it to RGB.
    if image_id[2:3] == "E":
        image = bayer_to_rgb(image)
    return image
    
def get_lab_image(image_id):        
    return color.rgb2lab(get_rgb_image(image_id))

In [4]:
top_image_id = "NRE_0015_0668275956_042ECM_N0030188NCAM00400_01_0LLJ"
bottom_image_id = "NRE_0015_0668275956_042ECM_N0030188NCAM00400_05_0LLJ"

top_lab = get_lab_image(top_image_id)
bottom_lab = get_lab_image(bottom_image_id)

tiles_by_origin = {
    (0, 0): top_lab,
    (0, 952): bottom_lab
}

tig = TileImageGrid(tiles_by_origin)

In [5]:
edge_of_top = tig.edge(0, 0, Edge.BOTTOM)
edge_of_bottom = tig.edge(0, 1, Edge.TOP)

In [6]:
def get_edge_pixels(edge):
    h, w, pix = edge.shape
    return edge.reshape((h * w, pix))

src = get_edge_pixels(edge_of_bottom)
targ = get_edge_pixels(edge_of_top)


In [7]:
x = src[:, 0]
y = src[:, 1]
z = src[:, 2]

d_edge = targ - src
u = d_edge[:, 0]
v = d_edge[:, 1]
w = d_edge[:, 2]


In [10]:
ipv.figure()
quiver = ipv.quiver(x, y, z, u, v, w, size=0.25)
ipv.pylab.xlim(-20, 120)
ipv.pylab.ylim(-130, 130)
ipv.pylab.zlim(-130, 130)
ipv.xyzlabel("L", "a", "b")
ipv.show()